## код model

In [1]:
"""
Full definition of a GPT Language Model, all of it in this single file.
References:
1) the official GPT-2 TensorFlow implementation released by OpenAI:
https://github.com/openai/gpt-2/blob/master/src/model.py
2) huggingface/transformers PyTorch implementation:
https://github.com/huggingface/transformers/blob/main/src/transformers/models/gpt2/modeling_gpt2.py
"""

import time as tm
t1_itogo = tm.time()
t1_temp = tm.time()

import math
import inspect
from dataclasses import dataclass

import torch
import torch.nn as nn
from torch.nn import functional as F

import transformers

import matplotlib.pyplot as plt
import numpy as np  # for normalization

C:\Users\cab\anaconda3\envs\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

"""
Sample from a trained model
"""
import os
import pickle
from contextlib import nullcontext
import tiktoken

t2_temp = tm.time()
print('импорт библиотек занял', round(t2_temp - t1_temp)//60,'минут', 
      round(t2_temp - t1_temp)%60,'секунд')

импорт библиотек занял 0 минут 18 секунд


In [3]:
t1_temp = tm.time()

# def normalize_image(im):   прошлая версия функции normalize_image, ChatGPT предложил другую (см. ниже) 
                             #Чтобы исключить ошибки при операцией над скалирным массивом или числом 
#    im_res = im - np.min(im)
#    return (1 / np.max(im_res)) * im_res

def normalize_image(im: np.ndarray) -> np.ndarray:
    """Normalize array to [0,1] using min-max. Safe for constant arrays."""
    im = np.asarray(im)
    im_res = im - np.min(im)
    maxv = np.max(im_res)
    if maxv == 0:
        return np.zeros_like(im_res)
    return (1.0 / maxv) * im_res

In [4]:
class LayerNorm(nn.Module):
    """ LayerNorm but with an optional bias. PyTorch doesn't support simply bias=False """

    def __init__(self, ndim, bias):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(ndim))  # LayerNorm объект класса nn.Module и мы тут явно задаем nn.Parameter
        self.bias = nn.Parameter(torch.zeros(ndim)) if bias else None

    def forward(self, input):
        return F.layer_norm(input, self.weight.shape, self.weight, self.bias, 1e-5)

In [5]:
class CausalSelfAttention(nn.Module):# ме не задаем тут nn.Parameters, потому что они есть во вложенных слоях класса nn.Linear

    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # key, query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd, bias=config.bias) # раз тут есть слой класса nn.Linear 
                                                                                # то в нем уже есть "место" для атрибута nn.Parameter
                                                                                # тут у нас тот самый линейный слой, который превращает 
                                                     #входной промпт x = тензор размера BxTxC в тензор размера BxTx3C
                                                     # Из него мы потом сможем вырезать Q, K и V, чтобы считать Attention
        self.c_proj = nn.Linear(config.n_embd, config.n_embd, bias=config.bias)
        # regularization
        self.attn_dropout = nn.Dropout(config.dropout)
        self.resid_dropout = nn.Dropout(config.dropout)
        self.n_head = config.n_head
        self.n_embd = config.n_embd # тут просто задается, что длина вектора  эмбеддинга (эмбеддинг = эмбеддинг токена) такая, 
                                    #  как задано в какой-то штуке config, которая подается на вход CausalSelfAttention
                                    # и с которой мы разберемся потом
        self.dropout = config.dropout
        # flash attention make GPU go brrrrr but support is only in PyTorch >= 2.0
        self.flash = hasattr(torch.nn.functional, 'scaled_dot_product_attention')
        if not self.flash:
            print("WARNING: using slow attention. Flash Attention requires PyTorch >= 2.0")
            # causal mask to ensure that attention is only applied to the left in the input sequence
            self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size))
                                        .view(1, 1, config.block_size, config.block_size))

    def plot_weight_matrices(self, transpose: bool = True, show_qkv: bool = True, cmap: str = "coolwarm", normalize: bool = True):
        """
        Визуализирует матрицы весов внимания слоя: c_attn и c_proj с помощью plt.imshow.
        - transpose=True: приводим к «математическому» виду (C×3C для c_attn, C×C для c_proj),
          т.к. в PyTorch у Linear weight хранится как (out_features, in_features).
        - show_qkv: если True, разрезаем c_attn на W_q, W_k, W_v и показываем их отдельно;
                    если False — показываем только целиком c_attn и c_proj.
        - cmap: палитра для imshow.
        Возвращает (fig, axes).
        """
        with torch.no_grad():
            # c_attn: (out=3C, in=C) в виде хранения PyTorch
            W_cat = self.c_attn.weight.detach().cpu()
            if transpose:
                W_cat = W_cat.T  # теперь (C, 3C)
            C = self.n_embd
            # разрезаем на три блока по последней оси (если transpose=True это столбцы; иначе строки)
            if transpose:
                W_q, W_k, W_v = W_cat[:, :C], W_cat[:, C:2*C], W_cat[:, 2*C:]
            else:
                W_q, W_k, W_v = W_cat[:C, :], W_cat[C:2*C, :], W_cat[2*C:, :]

            # c_proj: (out=C, in=C) в хранении PyTorch
            W_proj = self.c_proj.weight.detach().cpu()
            if transpose:
                W_proj = W_proj.T  # хотим (C, C)

            # подбираем общие vmin/vmax для сравнимой цветовой шкалы
            import numpy as np
            mats = [W_cat, W_proj] + ([W_q, W_k, W_v] if show_qkv else [])
            if normalize:
                W_proj = normalize_image(W_proj)
                if show_qkv:
                    W_q, W_k, W_v = normalize_image(W_q), normalize_image(W_k), normalize_image(W_v)
                else:
                    W_cat = normalize_image(W_cat)
                vmin, vmax = 0.0, 1.0
            else:
                vmax = max(float(np.abs(m).max()) for m in mats)
                vmin = -vmax

            if show_qkv:
                fig, axes = plt.subplots(1, 4, figsize=(18, 4))
                ims = []
                im = axes[0].imshow(W_proj, aspect='auto', interpolation='nearest', cmap=cmap, vmin=vmin, vmax=vmax)
                axes[0].set_title('c_proj weight')
                ims.append(im)
                im = axes[1].imshow(W_q, aspect='auto', interpolation='nearest', cmap=cmap, vmin=vmin, vmax=vmax)
                axes[1].set_title('c_attn: W_q')
                ims.append(im)
                im = axes[2].imshow(W_k, aspect='auto', interpolation='nearest', cmap=cmap, vmin=vmin, vmax=vmax)
                axes[2].set_title('c_attn: W_k')
                ims.append(im)
                im = axes[3].imshow(W_v, aspect='auto', interpolation='nearest', cmap=cmap, vmin=vmin, vmax=vmax)
                axes[3].set_title('c_attn: W_v')
                ims.append(im)
                for ax in axes:
                    ax.set_xlabel('in features')
                    ax.set_ylabel('out features')
                fig.colorbar(ims[0], ax=axes, fraction=0.02)
            else:
                fig, axes = plt.subplots(1, 2, figsize=(10, 4))
                im0 = axes[0].imshow(W_proj, aspect='auto', interpolation='nearest', cmap=cmap, vmin=vmin, vmax=vmax)
                axes[0].set_title('c_proj weight')
                im1 = axes[1].imshow(W_cat, aspect='auto', interpolation='nearest', cmap=cmap, vmin=vmin, vmax=vmax)
                axes[1].set_title('c_attn (concat Q|K|V)')
                for ax in axes:
                    ax.set_xlabel('in features')
                    ax.set_ylabel('out features')
                fig.colorbar(im0, ax=axes, fraction=0.02)
        return fig, axes
    # ⟦ADDED END⟧ конец метода plot_weight_matrices
    def forward(self, x):
        B, T, C = x.size()  # batch size, sequence length, embedding dim
                            # тут рассчитываем размерность промпта = размерность тензора с точки зрения обработки в CasualSelfAttention
                            #B = batch size = количество батчей. Для обученния (где model тоже используется), их обычно несколько
                            # чтобы учиться одновременно на разных промптах
        q, k, v = self.c_attn(x).split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)

    # ⬅️ Добавим это
        self.last_k = k.detach().cpu()
        self.last_q = q.detach().cpu()
        self.last_v = v.detach().cpu()

        if self.flash:
            y = torch.nn.functional.scaled_dot_product_attention(
            q, k, v, attn_mask=None,
            dropout_p=self.dropout if self.training else 0,
            is_causal=True)
        else:
            att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
            att = att.masked_fill(self.bias[:, :, :T, :T] == 0, float('-inf'))
            att = F.softmax(att, dim=-1)
            att = self.attn_dropout(att)
            y = att @ v

        y = y.transpose(1, 2).contiguous().view(B, T, C)
        y = self.resid_dropout(self.c_proj(y))
        return y


t2_temp = tm.time()
print('этот блок занял', round(t2_temp - t1_temp)//60,'минут', 
      round(t2_temp - t1_temp)%60,'секунд')

этот блок занял 0 минут 0 секунд


In [6]:
class MLP(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.c_fc    = nn.Linear(config.n_embd, 4 * config.n_embd, bias=config.bias)
        self.gelu    = nn.GELU()
        self.c_proj  = nn.Linear(4 * config.n_embd, config.n_embd, bias=config.bias)
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        x = self.dropout(x)
        return x

In [7]:
class Block(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.ln_1 = LayerNorm(config.n_embd, bias=config.bias)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = LayerNorm(config.n_embd, bias=config.bias)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x


In [8]:
@dataclass
class GPTConfig:
    block_size: int = 1024
    vocab_size: int = 50304 # GPT-2 vocab_size of 50257, padded up to nearest multiple of 64 for efficiency
    n_layer: int = 12
    n_head: int = 12
    n_embd: int = 768
    dropout: float = 0.0
    bias: bool = True # True: bias in Linears and LayerNorms, like GPT-2. False: a bit better and faster


In [9]:
class GPT(nn.Module):

    def __init__(self, config):
        super().__init__()
        assert config.vocab_size is not None
        assert config.block_size is not None
        self.config = config

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            wpe = nn.Embedding(config.block_size, config.n_embd),
            drop = nn.Dropout(config.dropout),
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = LayerNorm(config.n_embd, bias=config.bias),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        # with weight tying when using torch.compile() some warnings get generated:
        # "UserWarning: functional_call was passed multiple values for tied weights.
        # This behavior is deprecated and will be an error in future versions"
        # not 100% sure what this is, so far seems to be harmless. TODO investigate
        self.transformer.wte.weight = self.lm_head.weight # https://paperswithcode.com/method/weight-tying

        # init all weights
        self.apply(self._init_weights)
        # apply special scaled init to the residual projections, per GPT-2 paper
        for pn, p in self.named_parameters():
            if pn.endswith('c_proj.weight'):
                torch.nn.init.normal_(p, mean=0.0, std=0.02/math.sqrt(2 * config.n_layer))

        # report number of parameters
        print("number of parameters: %.2fM" % (self.get_num_params()/1e6,))

    def get_num_params(self, non_embedding=True):
        """
        Return the number of parameters in the model.
        For non-embedding count (default), the position embeddings get subtracted.
        The token embeddings would too, except due to the parameter sharing these
        params are actually used as weights in the final layer, so we include them.
        """
        n_params = sum(p.numel() for p in self.parameters())
        if non_embedding:
            n_params -= self.transformer.wpe.weight.numel()
        return n_params

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        device = idx.device
        b, t = idx.size()
        assert t <= self.config.block_size, f"Cannot forward sequence of length {t}, block size is only {self.config.block_size}"
        pos = torch.arange(0, t, dtype=torch.long, device=device) # shape (t)

        # forward the GPT model itself
        tok_emb = self.transformer.wte(idx) # token embeddings of shape (b, t, n_embd)
        pos_emb = self.transformer.wpe(pos) # position embeddings of shape (t, n_embd)
        x = self.transformer.drop(tok_emb + pos_emb)
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)

        if targets is not None:
            # if we are given some desired targets also calculate the loss
            logits = self.lm_head(x)
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)
        else:
            # inference-time mini-optimization: only forward the lm_head on the very last position
            logits = self.lm_head(x[:, [-1], :]) # note: using list [-1] to preserve the time dim
            loss = None

        return logits, loss

    def crop_block_size(self, block_size):
        # model surgery to decrease the block size if necessary
        # e.g. we may load the GPT2 pretrained model checkpoint (block size 1024)
        # but want to use a smaller block size for some smaller, simpler model
        assert block_size <= self.config.block_size
        self.config.block_size = block_size
        self.transformer.wpe.weight = nn.Parameter(self.transformer.wpe.weight[:block_size])
        for block in self.transformer.h:
            if hasattr(block.attn, 'bias'):
                block.attn.bias = block.attn.bias[:,:,:block_size,:block_size]

    @classmethod
    def from_pretrained(cls, model_type, override_args=None):
        assert model_type in {'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'}
        override_args = override_args or {} # default to empty dict
        # only dropout can be overridden see more notes below
        assert all(k == 'dropout' for k in override_args)
        from transformers import GPT2LMHeadModel
        print("loading weights from pretrained gpt: %s" % model_type)

        # n_layer, n_head and n_embd are determined from model_type
        config_args = {
            'gpt2':         dict(n_layer=12, n_head=12, n_embd=768),  # 124M params
            'gpt2-medium':  dict(n_layer=24, n_head=16, n_embd=1024), # 350M params
            'gpt2-large':   dict(n_layer=36, n_head=20, n_embd=1280), # 774M params
            'gpt2-xl':      dict(n_layer=48, n_head=25, n_embd=1600), # 1558M params
        }[model_type]
        print("forcing vocab_size=50257, block_size=1024, bias=True")
        config_args['vocab_size'] = 50257 # always 50257 for GPT model checkpoints
        config_args['block_size'] = 1024 # always 1024 for GPT model checkpoints
        config_args['bias'] = True # always True for GPT model checkpoints
        # we can override the dropout rate, if desired
        if 'dropout' in override_args:
            print(f"overriding dropout rate to {override_args['dropout']}")
            config_args['dropout'] = override_args['dropout']
        # create a from-scratch initialized minGPT model
        config = GPTConfig(**config_args)
        model = GPT(config)
        sd = model.state_dict()
        sd_keys = sd.keys()
        sd_keys = [k for k in sd_keys if not k.endswith('.attn.bias')] # discard this mask / buffer, not a param

        # init a huggingface/transformers model
        model_hf = GPT2LMHeadModel.from_pretrained(model_type)
        sd_hf = model_hf.state_dict()

        # copy while ensuring all of the parameters are aligned and match in names and shapes
        sd_keys_hf = sd_hf.keys()
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')] # ignore these, just a buffer
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.bias')] # same, just the mask (buffer)
        transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']
        # basically the openai checkpoints use a "Conv1D" module, but we only want to use a vanilla Linear
        # this means that we have to transpose these weights when we import them
        assert len(sd_keys_hf) == len(sd_keys), f"mismatched keys: {len(sd_keys_hf)} != {len(sd_keys)}"
        for k in sd_keys_hf:
            if any(k.endswith(w) for w in transposed):
                # special treatment for the Conv1D weights we need to transpose
                assert sd_hf[k].shape[::-1] == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k].t())
            else:
                # vanilla copy over the other parameters
                assert sd_hf[k].shape == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k])

        return model

    def configure_optimizers(self, weight_decay, learning_rate, betas, device_type):
        # start with all of the candidate parameters
        param_dict = {pn: p for pn, p in self.named_parameters()}
        # filter out those that do not require grad
        param_dict = {pn: p for pn, p in param_dict.items() if p.requires_grad}
        # create optim groups. Any parameters that is 2D will be weight decayed, otherwise no.
        # i.e. all weight tensors in matmuls + embeddings decay, all biases and layernorms don't.
        decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
        nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
        optim_groups = [
            {'params': decay_params, 'weight_decay': weight_decay},
            {'params': nodecay_params, 'weight_decay': 0.0}
        ]
        num_decay_params = sum(p.numel() for p in decay_params)
        num_nodecay_params = sum(p.numel() for p in nodecay_params)
        print(f"num decayed parameter tensors: {len(decay_params)}, with {num_decay_params:,} parameters")
        print(f"num non-decayed parameter tensors: {len(nodecay_params)}, with {num_nodecay_params:,} parameters")
        # Create AdamW optimizer and use the fused version if it is available
        fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
        use_fused = fused_available and device_type == 'cuda'
        extra_args = dict(fused=True) if use_fused else dict()
        optimizer = torch.optim.AdamW(optim_groups, lr=learning_rate, betas=betas, **extra_args)
        print(f"using fused AdamW: {use_fused}")

        return optimizer

    def estimate_mfu(self, fwdbwd_per_iter, dt):
        """ estimate model flops utilization (MFU) in units of A100 bfloat16 peak FLOPS """
        # first estimate the number of flops we do per iteration.
        # see PaLM paper Appendix B as ref: https://arxiv.org/abs/2204.02311
        N = self.get_num_params()
        cfg = self.config
        L, H, Q, T = cfg.n_layer, cfg.n_head, cfg.n_embd//cfg.n_head, cfg.block_size
        flops_per_token = 6*N + 12*L*H*Q*T
        flops_per_fwdbwd = flops_per_token * T
        flops_per_iter = flops_per_fwdbwd * fwdbwd_per_iter
        # express our flops throughput as ratio of A100 bfloat16 peak flops
        flops_achieved = flops_per_iter * (1.0/dt) # per second
        flops_promised = 312e12 # A100 GPU bfloat16 peak flops is 312 TFLOPS
        mfu = flops_achieved / flops_promised
        return mfu

    @torch.no_grad()
    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
        """
        Take a conditioning sequence of indices idx (LongTensor of shape (b,t)) and complete
        the sequence max_new_tokens times, feeding the predictions back into the model each time.
        Most likely you'll want to make sure to be in model.eval() mode of operation for this.
        """
        for _ in range(max_new_tokens):
            # if the sequence context is growing too long we must crop it at block_size
            idx_cond = idx if idx.size(1) <= self.config.block_size else idx[:, -self.config.block_size:]
            # forward the model to get the logits for the index in the sequence
            logits, _ = self(idx_cond)
            # pluck the logits at the final step and scale by desired temperature
            logits = logits[:, -1, :] / temperature
            # optionally crop the logits to only the top k options
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float('Inf')
            # apply softmax to convert logits to (normalized) probabilities
            probs = F.softmax(logits, dim=-1)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)
            # append sampled index to the running sequence and continue
            idx = torch.cat((idx, idx_next), dim=1)

        return idx


In [10]:
import torch

cfg = GPTConfig(block_size=8, vocab_size=64, n_layer=2, n_head=2, n_embd=16, dropout=0.0, bias=True)
m = GPT(cfg).eval()

x = torch.randint(0, cfg.vocab_size, (1, 8))
with torch.no_grad():
    logits, loss = m(x, x)

print("OK:", logits.shape, "loss:", float(loss))


number of parameters: 0.01M
OK: torch.Size([1, 8, 64]) loss: 3.923494338989258


In [11]:
def show_attn_weight_matrices(
    model: GPT, layer_idx: int, transpose: bool = True, show_qkv: bool = True, cmap: str = "coolwarm", normalize: bool = True):
    """
    Удобная обёртка: показываем weight-матрицы внимания (c_attn и c_proj) для блока с индексом layer_idx.
    Пример использования:
        fig, axes = show_attn_weight_matrices(model, layer_idx=0)
        plt.show()
    """
    attn = model.transformer.h[layer_idx].attn
    return attn.plot_weight_matrices(transpose=transpose, show_qkv=show_qkv, cmap=cmap, normalize=normalize)

In [12]:
# Проверяем generate
start_tok = torch.randint(0, cfg.vocab_size, (1, 1))  # один случайный токен-старт
print("Start token:", start_tok.tolist())

with torch.no_grad():
    out = m.generate(start_tok, max_new_tokens=10)  # сгенерируем 10 новых токенов

print("Generated sequence:", out.tolist())


Start token: [[12]]
Generated sequence: [[12, 55, 47, 49, 1, 54, 15, 47, 15, 19, 18]]


## Тут заканчивается код model и начинается код sample 

In [13]:
t1_temp = tm.time()

# -----------------------------------------------------------------------------
init_from = 'gpt2' # either 'resume' (from an out_dir) or a gpt2 variant (e.g. 'gpt2-xl')
out_dir = 'out' # ignored if init_from is not 'resume'
start_prompt = "quick_sort in Python is " # or "<|endoftext|>" or etc. Can also specify a file, use as: "FILE:prompt.txt"
num_samples = 1 # number of samples to draw
max_new_tokens = 1 # number of tokens generated in each sample
temperature = 0.8 # 1.0 = no change, < 1.0 = less random, > 1.0 = more random, in predictions
top_k = 2 # retain only the top_k most likely tokens, clamp others to have 0 probability
seed = 1337
device = 'cpu' # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1', etc.
dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16' # 'float32' or 'bfloat16' or 'float16'
compile = False # use PyTorch 2.0 to compile the model to be faster
#exec(open('configurator.py').read()) # overrides from command line or config file
# -----------------------------------------------------------------------------

t2_temp = tm.time()
print('при количестве создаваемых токенов ', max_new_tokens , 'и прогонов ', num_samples, 'этот блок занял', round(t2_temp - t1_temp)//60,'минут', 
      round(t2_temp - t1_temp)%60,'секунд')

при количестве создаваемых токенов  1 и прогонов  1 этот блок занял 0 минут 0 секунд


In [14]:
t1_temp = tm.time()

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

t2_temp = tm.time()
print('при количестве создаваемых токенов ', max_new_tokens , 'и прогонов ', num_samples, 'этот блок занял', round(t2_temp - t1_temp)//60,'минут', 
      round(t2_temp - t1_temp)%60,'секунд')

при количестве создаваемых токенов  1 и прогонов  1 этот блок занял 0 минут 0 секунд


In [15]:
t1_temp = tm.time()

# model
if init_from == 'resume':
    # init from a model saved in a specific directory
    ckpt_path = os.path.join(out_dir, 'ckpt.pt')
    checkpoint = torch.load(ckpt_path, map_location=device)
    gptconf = GPTConfig(**checkpoint['model_args'])
    model = GPT(gptconf)
    state_dict = checkpoint['model']
    unwanted_prefix = '_orig_mod.'
    for k,v in list(state_dict.items()):
        if k.startswith(unwanted_prefix):
            state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
    model.load_state_dict(state_dict)
elif init_from.startswith('gpt2'):
    # init from a given GPT-2 model
    model = GPT.from_pretrained(init_from, dict(dropout=0.0))

t2_temp = tm.time()
print('при количестве создаваемых токенов ', max_new_tokens , 'и прогонов ', num_samples, 'этот блок занял', round(t2_temp - t1_temp)//60,'минут', 
      round(t2_temp - t1_temp)%60,'секунд')

loading weights from pretrained gpt: gpt2
forcing vocab_size=50257, block_size=1024, bias=True
overriding dropout rate to 0.0
number of parameters: 123.65M
при количестве создаваемых токенов  1 и прогонов  1 этот блок занял 0 минут 17 секунд


In [16]:
t1_temp = tm.time()

model.eval()
model.to(device)
if compile:
    model = torch.compile(model) # requires PyTorch 2.0 (optional)

t2_temp = tm.time()
print('при количестве создаваемых токенов ', max_new_tokens , 'и прогонов ', num_samples, 'этот блок занял', round(t2_temp - t1_temp)//60,'минут', 
      round(t2_temp - t1_temp)%60,'секунд')

при количестве создаваемых токенов  1 и прогонов  1 этот блок занял 0 минут 0 секунд


In [17]:
t1_temp = tm.time()

# look for the meta pickle in case it is available in the dataset folder
load_meta = False
if init_from == 'resume' and 'config' in checkpoint and 'dataset' in checkpoint['config']: # older checkpoints might not have these...
    meta_path = os.path.join('data', checkpoint['config']['dataset'], 'meta.pkl')
    load_meta = os.path.exists(meta_path)
if load_meta:
    print(f"Loading meta from {meta_path}...")
    with open(meta_path, 'rb') as f:
        meta = pickle.load(f)
    # TODO want to make this more general to arbitrary encoder/decoder schemes
    stoi, itos = meta['stoi'], meta['itos']
    encode = lambda s: [stoi[c] for c in s]
    decode = lambda l: ''.join([itos[i] for i in l])
else:
    # ok let's assume gpt-2 encodings by default
    print("No meta.pkl found, assuming GPT-2 encodings...")
    enc = tiktoken.get_encoding("gpt2")
    encode = lambda s: enc.encode(s, allowed_special={"<|endoftext|>"})
    decode = lambda l: enc.decode(l)

t2_temp = tm.time()
print('при количестве создаваемых токенов ', max_new_tokens , 'и прогонов ', num_samples, 'этот блок занял', round(t2_temp - t1_temp)//60,'минут', 
      round(t2_temp - t1_temp)%60,'секунд')

No meta.pkl found, assuming GPT-2 encodings...
при количестве создаваемых токенов  1 и прогонов  1 этот блок занял 0 минут 0 секунд


In [23]:
t1_temp = tm.time()

# encode the beginning of the prompt
if start_prompt.startswith('FILE:'):
    with open(start_prompt[5:], 'r', encoding='utf-8') as f:
        start = f.read()

# >>> Добавляем отладочный вывод токенизации промпта
print("\n🧩 Токенизация промпта:")
print("Промпт:", repr(start_prompt))
print("Список ID:", start_ids)

try:
    # если ты используешь tiktoken, можно показать каждый токен и его байтовое представление
    for i, tid in enumerate(start_ids):
        bs = enc.decode_single_token_bytes(tid)
        try:
            frag = bs.decode("utf-8")
        except UnicodeDecodeError:
            frag = str(bs)
        safe = frag.replace(" ", "␠").replace("\n", "⏎")
        print(f"{i:>3} | id={tid:>5} | токен={safe!r}")
except Exception:
    # fallback для случая, если meta.pkl используется
    print("Токены:", [decode([tid]) for tid in start_ids])

start_ids = encode(start_prompt)
x = (torch.tensor(start_ids, dtype=torch.long, device=device)[None, ...])

# run generation
with torch.no_grad():
    with ctx:
        for k in range(num_samples):
            y = model.generate(x, max_new_tokens, temperature=temperature, top_k=top_k)
            print(decode(y[0].tolist()))
            print('---------------')

layer_idx = 0  # слой

#fig, axes = show_attn_weight_matrices(model, layer_idx=0)
#plt.show()


#q = show_attn_weight_matrices(model, layer_idx)



🧩 Токенизация промпта:
Промпт: 'quick_sort in Python is '
Список ID: [24209, 62, 30619, 287, 11361, 318, 220]
  0 | id=24209 | токен='quick'
  1 | id=   62 | токен='_'
  2 | id=30619 | токен='sort'
  3 | id=  287 | токен='␠in'
  4 | id=11361 | токен='␠Python'
  5 | id=  318 | токен='␠is'
  6 | id=  220 | токен='␠'
quick_sort in Python is  
---------------


In [19]:
# тут будем убирать комментирование при проверке в Lambda C
layer_idx = 0  # слой
head_idx = 0   # голова

attn = model.transformer.h[layer_idx].attn
# q = attn.last_q[0, head_idx]
#plt.imshow(q, aspect='auto', interpolation='nearest')
#plt.title(f"Layer {layer_idx}, Head {head_idx} — Q matrix")
#plt.colorbar()
#plt.show()

In [20]:
# Это просто проврочный блок, чтобы понять, что матрица формируется и ее можно показать
q = attn.last_q  # тензор формы (B, nh, T, hs)
print("Q shape:", q.shape)
print("Q[0,0] shape:", q[0, 0].shape)  # одна голова, один батч — форма (T, hs)
print("Q[0,0][:2]:", q[0, 0][:2])  # первые две строки, чтобы не ронять ядро

Q shape: torch.Size([1, 12, 7, 64])
Q[0,0] shape: torch.Size([7, 64])
Q[0,0][:2]: tensor([[ 6.3830e-01, -6.0561e-01,  1.5890e-01, -5.4816e-01,  3.3015e-01,
          5.5800e-01, -6.6338e-02, -1.0428e-01, -1.1624e+00,  5.6056e-01,
          5.6591e-01, -4.3868e-01, -5.0199e-01, -7.8386e-03, -1.4088e+00,
         -2.7537e-01, -9.3280e-02,  5.8579e-01, -1.8419e-01, -2.6414e-01,
         -1.3514e-01,  9.6321e-01, -1.5417e-01,  3.4709e-01, -5.3001e-02,
         -1.1317e+00, -5.9540e-01, -4.8682e-01, -1.6736e-01,  9.2002e-02,
          5.0135e-01,  4.0696e-01,  3.4328e-01, -3.8134e-03, -1.2630e+00,
         -1.7593e-01, -8.8416e-01,  2.3363e-01,  1.6483e-01,  8.1591e-01,
         -4.6211e-02,  2.3953e-01, -1.9078e-01,  3.9364e-01, -8.6423e-02,
          4.3528e-01,  6.4378e-01,  2.0167e-01,  2.5363e-01, -6.1410e-01,
         -4.8536e-01, -3.1194e-01,  2.7733e-01,  4.9332e-01,  1.8956e-01,
         -3.7899e-01, -1.7696e-01, -6.4536e-01, -1.1287e+00, -2.4175e-01,
          1.3164e-01,  2.0699e

In [21]:
layer_idx = 0  # слой
attn = model.transformer.h[layer_idx].attn  # attention слой 0

# Достаём матрицу весов (3C × C)
W_cat = attn.c_attn.weight.detach().cpu()  # shape: (3C, C)

# Транспонируем, чтобы получить "человеческий" формат: (C, 3C)
W_cat = W_cat.T  # shape: (C, 3C)

# Получаем размерность C
C = attn.n_embd

# Разрезаем на 3 части: W_q, W_k, W_v
W_q = W_cat[:, :C]
W_k = W_cat[:, C:2*C]
W_v = W_cat[:, 2*C:]

# Показываем только форму и пару строк (без печати всей матрицы!)
print("W_q shape:", W_q.shape)
print("W_k shape:", W_k.shape)
print("W_v shape:", W_v.shape)

print("W_q[:2, :5] =\n", W_q[:2, :5])
print("W_k[:2, :5] =\n", W_k[:2, :5])
print("W_v[:2, :5] =\n", W_v[:2, :5])


W_q shape: torch.Size([768, 768])
W_k shape: torch.Size([768, 768])
W_v shape: torch.Size([768, 768])
W_q[:2, :5] =
 tensor([[-0.4738, -0.2614, -0.0978, -0.3499,  0.2243],
        [ 0.0874,  0.1473,  0.2387,  0.2884,  0.6491]])
W_k[:2, :5] =
 tensor([[ 0.3660,  0.0771,  0.2226,  0.0138, -0.0494],
        [-0.4380, -0.1446, -0.4717, -0.0253,  0.0490]])
W_v[:2, :5] =
 tensor([[ 0.1421,  0.0329, -0.0667,  0.0188, -0.0645],
        [ 0.0162, -0.0633, -0.0636,  0.0340, -0.0322]])


In [22]:
t2_itogo = tm.time()
print('при количестве создаваемых токенов ', max_new_tokens , 'и прогонов ', num_samples, 'весь код занял', round(t2_itogo - t1_itogo)//60,'минут', 
      round(t2_itogo - t1_itogo)%60,'секунд')

при количестве создаваемых токенов  1 и прогонов  1 весь код занял 0 минут 35 секунд
